In [28]:
import math
import random
import sys
import sim as vrep
import numpy as np
import time
#import movearm
import polar_coordinates
print("modules imported")

modules imported


In [29]:
d=[[0.3968,-0.1949],[0.3668,0.15],[0.3668,0.495],[0.033,-0.1949],[0.033,0.15],[0.033,0.495]]
height= 0.1260


In [30]:
def move(handle, theta, clientID):
    if len(handle)!=len(theta) or len(handle)!=6:
        return -1

    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[1], 0, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[2], 0, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    time.sleep(2)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[0], theta[0]/180*np.pi, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    #time.sleep(3)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[3], theta[3]/180*np.pi, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    #time.sleep(3)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[4], theta[4]/180*np.pi, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    #time.sleep(3)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[5], theta[5]/180*np.pi, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    #time.sleep(3)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[1], theta[1]/180*np.pi, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)

    time.sleep(2)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, handle[2], theta[2]/180*np.pi, vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
  
    return 0
def hit(hammer, clientID):
    time.sleep(1.5)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, hammer, 0.03,vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    time.sleep(1.5)
    vrep.simxPauseCommunication(clientID,True)
    vrep.simxSetJointTargetPosition(clientID, hammer,  0,vrep.simx_opmode_oneshot) 
    vrep.simxPauseCommunication(clientID,False)
    return 0

In [31]:
def invkinematics(x,y,t,z = height):
    #define length parameters
    L1 = 0.152
    L2 = 0.120
    L3 = 0.24367
    L4 = 0.093
    L5 = 0.21324
    L6 = 0.083
    L7 = 0.08535
    L8 = 0.082
    L9 = 0.0535
    L10= 0.059
    
    Zcen = z - 0.01
    Xcen = x + 0.15
    Ycen = y - 0.15
    
    theta1 = 90. - np.arctan2(Xcen,Ycen)/np.pi*180 - np.arcsin((L6+0.027)/(Xcen**2+Ycen**2)**0.5)/np.pi*180
    y3end = Ycen - np.sin(theta1/180*np.pi)*L7 - np.cos(theta1/180*np.pi)*(L6+0.027)
    x3end = Xcen - np.cos(theta1/180*np.pi)*L7 + np.sin(theta1/180*np.pi)*(L6+0.027)
    z3end = Zcen + L10 + L8
    
    theta6 =  t-theta1
    theta5 = -90.
    L = (x3end**2+y3end**2+(z3end-L1)**2)**0.5
    theta3 = 180 - np.arccos((L3**2+L5**2-L**2)/2/L3/L5)*180/np.pi
    theta2 = -np.arcsin((z3end-L1)/L)/np.pi*180 - np.arcsin(np.sin(np.pi-theta3*np.pi/180)*L5/L)/np.pi*180
    theta4 = - theta3 - theta2
    return [theta1,theta2+90,theta3,theta4,theta5,-theta6]

In [33]:
invkinematics(0.2535, 0.15, -180.0)

[-15.819911516700518,
 29.189725574720015,
 91.15409103780354,
 -30.343816612523554,
 -90.0,
 164.18008848329947]

In [32]:
def end_pos(ball,hole):
    x_diff = ball[0]-hole[0]
    y_diff = ball[1]-hole[1]
    dis = (x_diff**2+y_diff**2)**0.5
    x_ret = 0.0535*x_diff/dis
    y_ret = 0.0535*y_diff/dis
    yaw = 360*math.atan2(-y_diff, -x_diff)/(2*np.pi)
    #whlie(yaw<0 or yaw>360)
    return [ball[0]+x_ret,ball[1]+y_ret, yaw]

In [35]:
end_pos([0.2,0.15],d[4])

[0.2535, 0.15, -180.0]

In [33]:
def checkin(ball_handle):
    #p = vrep.simxGetObjectPosition(clientID, ball, -1, vrep.simx_opmode_blocking)
    popy=[]
    ball_pos=[]
    for i in ball_handle:
        error, ball_p = vrep.simxGetObjectPosition(clientID, i, -1, vrep.simx_opmode_blocking)
        ball_pos.append(ball_p)
    #print(ball_pos)
    for i in range(len(ball_handle)):
        if(ball_pos[i][2]<0.125):
            error=vrep.simxRemoveObject(clientID,ball_handle[i],vrep.simx_opmode_oneshot)
            popy.append(i)
    popy.reverse()
    for p in popy:
        ball_handle.pop(p)
        ball_pos.pop(p)
    return ball_handle, ball_pos

In [52]:
def end_pos(ball,hole):
    x_diff = ball[0]-hole[0]
    y_diff = ball[1]-hole[1]
    dis = (x_diff**2+y_diff**2)**0.5
    x_ret = 0.0535*x_diff/dis
    y_ret = 0.0535*y_diff/dis
    yaw = 360*math.atan2(-y_diff, -x_diff)/(2*np.pi)
    return [ball[0]+x_ret,ball[1]+y_ret, yaw]

def sort_holes(holes, target):
    hole_distance = []
    sorted_holes = []
    for i in range(len(holes)):
        dis = ( (target[0]-holes[i][0])**2 + (target[1]-holes[i][1])**2 )**0.5
        hole_distance.append(dis)
        hole_num = []
    dis_del = []
    dis_del.extend(hole_distance)
    for i in range(len(holes)):
        min_d = min(dis_del)
        for j in range(len(hole_distance)):
            if (hole_distance[j] == min_d and not(j in(hole_num))):
                hole_num.append(j)
                break
        for j in range(len(dis_del)):
            if (dis_del[j] == min_d):
                del dis_del[j]
                break
    for i in range(len(hole_num)):
        sorted_holes.append(holes[hole_num[i]])
    return sorted_holes

def at_edge(ball, holes):
    rest_holes = []
    for i in range(len(holes)):
        x_diff = ball[0]-holes[i][0]
        y_diff = ball[1]-holes[i][1]
        dis = (x_diff**2+y_diff**2)**0.5
        x_ret = ball[0] + 0.057*x_diff/dis
        y_ret = ball[1] + 0.057*y_diff/dis
        yaw = math.atan2(-y_diff, -x_diff)
        x1 = x_ret + 0.005 * np.cos(yaw+np.pi/2)
        y1 = y_ret + 0.005 * np.sin(yaw+np.pi/2)
        x2 = x_ret - 0.005 * np.cos(yaw+np.pi/2)
        y2 = y_ret - 0.005 * np.sin(yaw+np.pi/2)
        if (x1 < 0.3705 and x1 > 0.0295 and y1 < 0.498 and y1 > -0.198 and x2 < 0.3705 and x2 > 0.0295 and y2 < 0.498 and y2 > -0.198):
            rest_holes.append(holes[i])
    return rest_holes

def obstacle_test(ball, balls, hole):
    x_diff = ball[0]-hole[0]
    y_diff = ball[1]-hole[1]
    A = -y_diff/x_diff
    C = -A*ball[0] - ball[1]
    yaw = 180*math.atan2(-y_diff, -x_diff)/np.pi
    for i in range(len(balls)):
        if (balls[i] == ball):
            continue
        a_BB = 180*math.atan2(balls[i][1]-ball[1], balls[i][0]-ball[0])/np.pi
        B_L = (balls[i][0] * A + balls[i][1] + C)/(A**2 + 1)**0.5
        if (abs(B_L) <= 0.02):
            if (-90 <= yaw and yaw <= 90):
                if (a_BB > yaw - 90 and a_BB < yaw +90):
                    print(balls[i])
                    return 1
            elif (yaw < -90):
                if (a_BB > yaw + 270 or a_BB < yaw + 90):
                    print(balls[i])
                    return 1
            elif (yaw > 90):
                if (a_BB > yaw - 90 or a_BB < yaw - 270):
                    print(balls[i])
                    return 1
    return 0 #when return 0, there will be no collision

def clear_test(target, end_pos, balls):
    mid_pos = [(end_pos[0]+target[0])/2, (end_pos[1]+target[1])/2]
    for ball in balls:
        if (ball != target):
            if ((ball[0] - mid_pos[0])**2+(ball[1] - mid_pos[1])**2 <= 0.03**2):
                return 1
    return 0

def hole_choose(balls, holes, target):
    sorted_holes = sort_holes(holes, target)
    rest_holes = at_edge(target, sorted_holes)
    target_hole = []
    for hole in rest_holes:
        pos = end_pos(target, hole)
        if (clear_test(target, pos, balls)):
            continue
        if (obstacle_test(target, balls, hole)):
            continue
        return hole
    return[0,0]

In [35]:
def overlep(arr,ball):
    for pos in arr:
        if(((pos[0]-ball[0])**2+(pos[1]-ball[1])**2)<0.0004):
            return 0
    return 1    

# Final Demo

In [61]:
"""
Created on Thu Mar 19 16:24:51 2020

@author: my13
"""
N=15

vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19997,True,True,5000,5)
print(clientID) # if 1, then we are connected.
if clientID!=-1:
    print("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)




#################get handle##################
ball_handle=[]
for i in range(N):
    error, ball = vrep.simxGetObjectHandle(clientID,"Sphere"+str(i), vrep.simx_opmode_blocking)
    ball_handle.append(ball)
pos=[]
while(len(pos)<N):
    temp = [random.uniform(0.0650,0.335),random.uniform(-0.158,0.458),0.1260]
    if(overlep(pos,temp)==1):
        pos.append(temp)
#print(pos)
for i in range(N):
    error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, pos[i], vrep.simx_opmode_blocking)
    #print(i)
time.sleep(5)
#get the handle and position of the ball

ball_pos = []
for i in range(N):
    error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)

    ball_pos.append(ball_p)
#print(ball_pos)
#handle the joints
joint=[]
for i in range(6):
    err_code,S = vrep.simxGetObjectHandle(clientID,"UR3_joint"+str(i+1), vrep.simx_opmode_blocking)
    #print(error)
    joint.append(S)   
#handle the hammer
error, hammer = vrep.simxGetObjectHandle(clientID,"hammer", vrep.simx_opmode_blocking)
#print(ball_pos)
##############start hit balls################

#get the cen position
while(len(ball_pos)!=0):
    target_hole=hole_choose(ball_pos, d, ball_pos[0])
    if(target_hole==[0,0]):
        temp1=ball_pos.pop(0)
        temp2=ball_handle.pop(0)
        ball_pos.append(temp1)
        ball_handle.append(temp2)
        continue
    poscen = end_pos([ball_pos[0][0],ball_pos[0][1]], target_hole)
    #print(poscen)
    #get the joint angles with reverse k
    thetas=invkinematics(poscen[0],poscen[1],poscen[2])
    #move the arm to the shooting location
    #print(thetas)
    move(joint, thetas, clientID)
    hit(hammer, clientID)
    time.sleep(0.5)
    move(joint, [0,0,0,0,0,0], clientID)
    time.sleep(0.5)
    for i in range(len(ball_handle)):
        error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)
        ball_pos[i] = ball_p
    for i in range(len(ball_handle)):
        error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, ball_pos[i], vrep.simx_opmode_blocking)

    ball_handle, ball_pos=checkin(ball_handle)

0
Connected to remote API server


KeyboardInterrupt: 

# Test case

In [64]:
N=1

vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19997,True,True,5000,5)
print(clientID) # if 1, then we are connected.
if clientID!=-1:
    print("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)
time.sleep(7)
ball_handle=[]
for i in range(N):
    error, ball = vrep.simxGetObjectHandle(clientID,"Sphere"+str(i), vrep.simx_opmode_blocking)
    ball_handle.append(ball)
pos=[]
while(len(pos)<N):
    temp = [0.0950,-0.125,0.1260]
    if(overlep(pos,temp)==1):
        pos.append(temp)

for i in range(N):
    error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, pos[i], vrep.simx_opmode_blocking)
    #print(i)
time.sleep(5)
#get the handle and position of the ball

ball_pos = []
for i in range(N):
    error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)

    ball_pos.append(ball_p)
#print(ball_pos)
#handle the joints
joint=[]
for i in range(6):
    err_code,S = vrep.simxGetObjectHandle(clientID,"UR3_joint"+str(i+1), vrep.simx_opmode_blocking)
    #print(error)
    joint.append(S)   
#handle the hammer
error, hammer = vrep.simxGetObjectHandle(clientID,"hammer", vrep.simx_opmode_blocking)
#print(ball_pos)
##############start hit balls################

#get the cen position
while(len(ball_pos)!=0):
    target_hole=hole_choose(ball_pos, d, ball_pos[0])
    if(target_hole==[0,0]):
        temp1=ball_pos.pop(0)
        temp2=ball_handle.pop(0)
        ball_pos.append(temp1)
        ball_handle.append(temp2)
        continue
    poscen = end_pos([ball_pos[0][0],ball_pos[0][1]], target_hole)
    #print(poscen)
    #get the joint angles with reverse k
    thetas=invkinematics(poscen[0],poscen[1],poscen[2])
    #move the arm to the shooting location
    #print(thetas)
    move(joint, thetas, clientID)
    hit(hammer, clientID)
    time.sleep(0.5)
    move(joint, [0,0,0,0,0,0], clientID)
    time.sleep(0.5)
    for i in range(len(ball_handle)):
        error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)
        ball_pos[i] = ball_p
    for i in range(len(ball_handle)):
        error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, ball_pos[i], vrep.simx_opmode_blocking)

    ball_handle, ball_pos=checkin(ball_handle)

0
Connected to remote API server


In [40]:
N=2

vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19997,True,True,5000,5)
print(clientID) # if 1, then we are connected.
if clientID!=-1:
    print("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

ball_handle=[]
for i in range(N):
    error, ball = vrep.simxGetObjectHandle(clientID,"Sphere"+str(i), vrep.simx_opmode_blocking)
    ball_handle.append(ball)
pos=[[0.12,-0.1,0.126],[0.15,-0.075,0.126]]


for i in range(N):
    error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, pos[i], vrep.simx_opmode_blocking)
    #print(i)
time.sleep(5)
#get the handle and position of the ball

ball_pos = []
for i in range(N):
    error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)

    ball_pos.append(ball_p)
#print(ball_pos)
#handle the joints
joint=[]
for i in range(6):
    err_code,S = vrep.simxGetObjectHandle(clientID,"UR3_joint"+str(i+1), vrep.simx_opmode_blocking)
    #print(error)
    joint.append(S)   
#handle the hammer
error, hammer = vrep.simxGetObjectHandle(clientID,"hammer", vrep.simx_opmode_blocking)
#print(ball_pos)
##############start hit balls################

#get the cen position
while(len(ball_pos)!=0):
    target_hole=hole_choose(ball_pos, d, ball_pos[0])
    if(target_hole==[0,0]):
        temp1=ball_pos.pop(0)
        temp2=ball_handle.pop(0)
        ball_pos.append(temp1)
        ball_handle.append(temp2)
        continue
    poscen = end_pos([ball_pos[0][0],ball_pos[0][1]], target_hole)
    #print(poscen)
    #get the joint angles with reverse k
    thetas=invkinematics(poscen[0],poscen[1],poscen[2])
    #move the arm to the shooting location
    #print(thetas)
    move(joint, thetas, clientID)
    hit(hammer, clientID)
    time.sleep(0.5)
    move(joint, [0,0,0,0,0,0], clientID)
    time.sleep(0.5)
    for i in range(len(ball_handle)):
        error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)
        ball_pos[i] = ball_p
    for i in range(len(ball_handle)):
        error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, ball_pos[i], vrep.simx_opmode_blocking)

    ball_handle, ball_pos=checkin(ball_handle)

0
Connected to remote API server
not clear
not clear
not clear


In [19]:
N=2

vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19997,True,True,5000,5)
print(clientID) # if 1, then we are connected.
if clientID!=-1:
    print("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

ball_handle=[]
for i in range(N):
    error, ball = vrep.simxGetObjectHandle(clientID,"Sphere"+str(i), vrep.simx_opmode_blocking)
    ball_handle.append(ball)
pos=[[0.12,-0.1,0.126],[0.05, -0.175, 0.126]]


for i in range(N):
    error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, pos[i], vrep.simx_opmode_blocking)
    #print(i)
time.sleep(5)
#get the handle and position of the ball

ball_pos = []
for i in range(N):
    error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)

    ball_pos.append(ball_p)
#print(ball_pos)
#handle the joints
joint=[]
for i in range(6):
    err_code,S = vrep.simxGetObjectHandle(clientID,"UR3_joint"+str(i+1), vrep.simx_opmode_blocking)
    #print(error)
    joint.append(S)   
#handle the hammer
error, hammer = vrep.simxGetObjectHandle(clientID,"hammer", vrep.simx_opmode_blocking)
#print(ball_pos)
##############start hit balls################

#get the cen position
while(len(ball_pos)!=0):
    target_hole=hole_choose(ball_pos, d, ball_pos[0])
    if(target_hole==[0,0]):
        temp1=ball_pos.pop(0)
        temp2=ball_handle.pop(0)
        ball_pos.append(temp1)
        ball_handle.append(temp2)
        continue
    poscen = end_pos([ball_pos[0][0],ball_pos[0][1]], target_hole)
    #print(poscen)
    #get the joint angles with reverse k
    thetas=invkinematics(poscen[0],poscen[1],poscen[2])
    #move the arm to the shooting location
    #print(thetas)
    move(joint, thetas, clientID)
    hit(hammer, clientID)
    time.sleep(0.5)
    move(joint, [0,0,0,0,0,0], clientID)
    time.sleep(0.5)
    for i in range(len(ball_handle)):
        error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)
        ball_pos[i] = ball_p
    for i in range(len(ball_handle)):
        error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, ball_pos[i], vrep.simx_opmode_blocking)

    ball_handle, ball_pos=checkin(ball_handle)

0
Connected to remote API server
[0.04999807849526405, -0.1750042736530304, 0.1260499209165573]
obs


In [25]:
N=7

vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19997,True,True,5000,5)
print(clientID) # if 1, then we are connected.
if clientID!=-1:
    print("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

ball_handle=[]
for i in range(N):
    error, ball = vrep.simxGetObjectHandle(clientID,"Sphere"+str(i), vrep.simx_opmode_blocking)
    ball_handle.append(ball)
pos=[[0.2,-0.14,0.126],[0.05, -0.175, 0.126],[0.35, -0.175, 0.126],[0.35, 0.15, 0.126],[0.05, 0.15, 0.126],[0.35, 0.475, 0.126],[0.05, 0.475, 0.126]]


for i in range(N):
    error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, pos[i], vrep.simx_opmode_blocking)
    #print(i)
time.sleep(5)
#get the handle and position of the ball

ball_pos = []
for i in range(N):
    error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)

    ball_pos.append(ball_p)
#print(ball_pos)
#handle the joints
joint=[]
for i in range(6):
    err_code,S = vrep.simxGetObjectHandle(clientID,"UR3_joint"+str(i+1), vrep.simx_opmode_blocking)
    #print(error)
    joint.append(S)   
#handle the hammer
error, hammer = vrep.simxGetObjectHandle(clientID,"hammer", vrep.simx_opmode_blocking)
#print(ball_pos)
##############start hit balls################

#get the cen position
while(len(ball_pos)!=0):
    target_hole=hole_choose(ball_pos, d, ball_pos[0])
    if(target_hole==[0,0]):
        temp1=ball_pos.pop(0)
        temp2=ball_handle.pop(0)
        ball_pos.append(temp1)
        ball_handle.append(temp2)
        continue
    poscen = end_pos([ball_pos[0][0],ball_pos[0][1]], target_hole)
    #print(poscen)
    #get the joint angles with reverse k
    thetas=invkinematics(poscen[0],poscen[1],poscen[2])
    #move the arm to the shooting location
    #print(thetas)
    move(joint, thetas, clientID)
    hit(hammer, clientID)
    time.sleep(0.5)
    move(joint, [0,0,0,0,0,0], clientID)
    time.sleep(0.5)
    for i in range(len(ball_handle)):
        error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)
        ball_pos[i] = ball_p
    for i in range(len(ball_handle)):
        error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, ball_pos[i], vrep.simx_opmode_blocking)

    ball_handle, ball_pos=checkin(ball_handle)

0
Connected to remote API server
[0.04999730363488197, -0.1750054657459259, 0.1260499209165573]
obs
[0.3491206169128418, -0.17589429020881653, 0.1260499209165573]
obs
[0.3500310778617859, 0.1499803513288498, 0.1260499209165573]
obs
[0.05034255236387253, 0.15006954967975616, 0.1260499209165573]
obs
[0.3492516279220581, 0.4758123755455017, 0.1260499209165573]
obs
[0.05000000074505806, 0.4749999940395355, 0.1260499209165573]
obs


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in arccos


KeyboardInterrupt: 

In [53]:
N=3

vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19997,True,True,5000,5)
print(clientID) # if 1, then we are connected.
if clientID!=-1:
    print("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

ball_handle=[]
for i in range(N):
    error, ball = vrep.simxGetObjectHandle(clientID,"Sphere"+str(i), vrep.simx_opmode_blocking)
    ball_handle.append(ball)
pos=[[0.2,-0.17,0.126],[0.05, -0.175, 0.126],[0.35, -0.175, 0.126]]


for i in range(N):
    error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, pos[i], vrep.simx_opmode_blocking)
    #print(i)
time.sleep(5)
#get the handle and position of the ball

ball_pos = []
for i in range(N):
    error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)

    ball_pos.append(ball_p)
#print(ball_pos)
#handle the joints
joint=[]
for i in range(6):
    err_code,S = vrep.simxGetObjectHandle(clientID,"UR3_joint"+str(i+1), vrep.simx_opmode_blocking)
    #print(error)
    joint.append(S)   
#handle the hammer
error, hammer = vrep.simxGetObjectHandle(clientID,"hammer", vrep.simx_opmode_blocking)
#print(ball_pos)
##############start hit balls################

#get the cen position
while(len(ball_pos)!=0):
    target_hole=hole_choose(ball_pos, d, ball_pos[0])
    if(target_hole==[0,0]):
        temp1=ball_pos.pop(0)
        temp2=ball_handle.pop(0)
        ball_pos.append(temp1)
        ball_handle.append(temp2)
        continue
    poscen = end_pos([ball_pos[0][0],ball_pos[0][1]], target_hole)
    #print(poscen)
    #get the joint angles with reverse k
    thetas=invkinematics(poscen[0],poscen[1],poscen[2])
    #move the arm to the shooting location
    #print(thetas)
    move(joint, thetas, clientID)
    hit(hammer, clientID)
    time.sleep(0.5)
    move(joint, [0,0,0,0,0,0], clientID)
    time.sleep(0.5)
    for i in range(len(ball_handle)):
        error, ball_p = vrep.simxGetObjectPosition(clientID, ball_handle[i], -1, vrep.simx_opmode_blocking)
        ball_pos[i] = ball_p
    for i in range(len(ball_handle)):
        error = vrep.simxSetObjectPosition(clientID, ball_handle[i], -1, ball_pos[i], vrep.simx_opmode_blocking)

    ball_handle, ball_pos=checkin(ball_handle)

0
Connected to remote API server
[0.05000052973628044, -0.17499980330467224, 0.1260499209165573]
[0.3497311472892761, -0.17530016601085663, 0.1260499209165573]


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in arccos


KeyboardInterrupt: 